In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, KernelPCA, FastICA
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import os
import spacy
!pip install plotnine 


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\jaylo\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [60]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, KernelPCA, FastICA
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import os
import spacy 
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from PIL import Image, ImageDraw
import plotnine as gg
from plotnine import *
from PIL import Image, ImageDraw, ImageChops
from sklearn.cluster import KMeans
from PIL import Image, ImageDraw
import numpy as np
from matplotlib.colors import ListedColormap
from matplotlib.patches import Circle
import numpy as np
from scipy.spatial.distance import pdist, squareform
from tqdm import tqdm 

def preprocess_data(text,data_name):
    

    # Function to lemmatize text using spaCy
    def lemmatize_text(text):
        doc = nlp(text)
        return " ".join([token.lemma_ for token in doc])
        # Load spaCy model
        #nlp = spacy.load("en_core_web_sm")
     # Load the data
    data = pd.read_csv(data_name)
    
    
    # Extract character names and remove from main data
    ids = data.iloc[:, 0].tolist()
    print(ids)
    main_data = data.iloc[:, 2:]

    # Identify numeric and text columns
    numeric_columns = main_data.select_dtypes(include=[np.number]).columns
    text_columns = main_data.select_dtypes(exclude=[np.number]).columns
    print(text_columns)
    # Preprocess numeric columns
    #scaler = StandardScaler()
    #imputer = SimpleImputer(strategy='mean')
    #numeric_data = pd.DataFrame(scaler.fit_transform(imputer.fit_transform(main_data[numeric_columns])), 
    #                           columns=numeric_columns)
   

    # Preprocess text columns using TfidfVectorizer and spaCy lemmatization
    if text:
        tfidf = TfidfVectorizer()  # You can adjust max_features as needed
        text_data = main_data[text_columns].fillna('')
        text_data_combined = text_data.apply(lambda x: ' '.join(x), axis=1)
        print(text_data_combined)
        #lemmatized_text = text_data_combined.apply(lemmatize_text)
        tfidf_matrix = tfidf.fit_transform(text_data_combined)
        tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf.get_feature_names_out())
        processed_data = pd.concat([tfidf_df], axis=1)

    


    return [main_data,ids]

from sklearn.cluster import KMeans
from PIL import Image, ImageDraw
import numpy as np
from matplotlib.colors import ListedColormap
from matplotlib.patches import Circle

def crop_to_circle(image):
    # Convert to RGBA if not already
    image = image.convert('RGBA')
    
    # Create a circular mask
    mask = Image.new('L', image.size, 0)
    draw = ImageDraw.Draw(mask)
    draw.ellipse((0, 0) + image.size, fill=255)
    
    # Get the alpha channel from the original image
    original_alpha = image.split()[3]
    
    # Combine the circular mask with the original alpha channel
    new_alpha = ImageChops.multiply(mask, original_alpha)
    
    # Create a new image with the modified alpha channel
    r, g, b, _ = image.split()
    output = Image.merge('RGBA', (r, g, b, new_alpha))
    
    return output

def apply_force_spacing(positions, names, min_distance=1.0, iterations=50, force_strength=0.1):
    """
    Apply force-directed spacing to ensure elements aren't too close.
    """
    # Normalize positions first
    
    
    for _ in tqdm(range(iterations), desc="Adjusting positions"):
        distances = squareform(pdist(positions))
        
        for i in range(len(positions)):
            too_close = np.where(distances[i] < min_distance)[0]
            too_close = too_close[too_close != i]
            
            if len(too_close) > 0:
                for j in too_close:
                    diff = positions[i] - positions[j]
                    length = np.linalg.norm(diff)
                    if length > 0:
                        diff = diff / length
                        force = force_strength * (min_distance - length)
                        positions[i] += diff * force
                        positions[j] -= diff * force
    
    return positions

def get_column(list_of_lists, column_index):
  """
  Returns a column from a list of lists.

  Args:
    list_of_lists: The list of lists.
    column_index: The index of the column to get.

  Returns:
    A list containing the elements of the specified column.
  """
  return [row[column_index] for row in list_of_lists]

def decompose_data(processed_data, id_label_data):
    def get_image(path, zoom=0.1):
        img = plt.imread(path)
        if isinstance(img, np.ndarray):
            # Convert to PIL Image
            img = Image.fromarray((img * 255).astype(np.uint8))
        img = crop_to_circle(img)
        return OffsetImage(img, zoom=zoom)

   
    
    ids = id_label_data

  
        # 2D decomposition
    tsne2 = PCA(n_components=2)
    processed_data= tsne2.fit_transform(processed_data)

    
    # Apply force-directed spacing with more reasonable parameters
    spaced_positions = apply_force_spacing(
        processed_data,
        ids,
        min_distance=7,  # Now works with normalized data
        iterations=1000,
        force_strength=0.0001
    )

    # Clustering on the spaced positions
    kmeans_2d = KMeans(n_clusters=5, random_state=42)
    clusters_2d = kmeans_2d.fit_predict(spaced_positions)
    


    colors = [
        '#BF0D3E', 
        '#1B4D89', 
        '#006B54',  
        '#7B2CBF',  
        '#CC5500',  
        #'#116466', 
        #'#9B2226',  
        #'#354F52',
        #'#6B4423',  
        #'#693C72' 
    ]
    # Create the graph
    plt.figure(figsize=(5, 4))
    
    
    # Plot colored circles for each cluster
    for cluster in range(5):  # Changed to 8 clusters
        mask = clusters_2d == cluster
        plt.scatter(spaced_positions[mask, 0], spaced_positions[mask, 1], 
                   color=colors[cluster], alpha=0.7, s=300)

    # Add character images
    for i, character in enumerate(ids):
        try:
            img_path = os.path.join('images', f"{character}.png")
            ab = AnnotationBbox(get_image(img_path), 
                              (spaced_positions[i, 0], spaced_positions[i, 1]), 
                              frameon=False)
            plt.gca().add_artist(ab)
        except FileNotFoundError:
            print(f"Image not found for {character}")

    plt.suptitle('Pokemon Unite Meta (April 27 2025)')
    # Add legend for clusters
    #legend_elements = [plt.Line2D([0], [0], marker='o', color='w', 
    #                             markerfacecolor=color, 
    #                             label=f'Cluster {i+1}') 
    #                  for i, color in enumerate(colors)]
    #plt.legend(handles=legend_elements, loc='upper right')
    
    plt.tight_layout()
    plt.xlabel('Play-Rate (Mix of Pick and Ban Rate)',fontsize=8)
    plt.ylabel('Win-Rate',fontsize=10)
    plt.savefig('clustering_image.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    print("Processing complete. Results saved to 'data-1d.csv', 'data-2d.csv' and 'TSNE_clustering_images.png'.")

df,chars = preprocess_data(False,"data.csv")
decompose_data(df,chars)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74]
Index([], dtype='object')


Adjusting positions: 100%|██████████| 1000/1000 [00:14<00:00, 71.01it/s]


Processing complete. Results saved to 'data-1d.csv', 'data-2d.csv' and 'TSNE_clustering_images.png'.


In [48]:

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, KernelPCA
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
data = pd.read_csv("Pokemon Data.csv")
# Drop the first column and normalize the remaining data
labels = data.iloc[:,0]
data_values = data.drop(columns=['Pokemon'])
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_values)

# Perform PCA with 2 components
pca = PCA(n_components=2)
pca_data = pca.fit_transform(scaled_data)

# Perform Kernel PCA with 2 components
kpca = KernelPCA(n_components=2, kernel='rbf')
kpca_data = kpca.fit_transform(scaled_data)

# Convert PCA and KPCA results to dataframes for easy handling
pca_df = pd.DataFrame(pca_data, columns=['PC1', 'PC2'])
kpca_df = pd.DataFrame(kpca_data, columns=['KPCA1', 'KPCA2'])

# Display the first few rows of the PCA and KPCA results
(labels.head(),pca_df.head(), kpca_df.head())

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Elbow method to determine the optimal number of clusters
def elbow_method(data):
    sse = []
    for k in range(1, 11):
        kmeans = KMeans(n_clusters=k, random_state=0)
        kmeans.fit(data)
        sse.append(kmeans.inertia_)
    return sse

# Plot the elbow method for PCA data
reg_sse = elbow_method(scaled_data)
pca_sse = elbow_method(pca_df)
kpca_sse = elbow_method(kpca_df)

plt.figure(figsize=(12, 6), dpi=300)
plt.plot(range(1, 11), reg_sse, marker='p', label='Raw')
plt.plot(range(1, 11), pca_sse, marker='o', label='PCA')
plt.plot(range(1, 11), kpca_sse, marker='s', label='KPCA')

plt.xlabel('Number of clusters',fontsize=8)
plt.ylabel('Sum of squared distances',fontsize=8)
plt.title('Elbow Method for Optimal Number of Clusters',fontsize=12)
plt.legend()
plt.grid(True)
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'Pokemon Data.csv'

In [47]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, KernelPCA
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import DataFrame
import random
data = pd.read_csv("raw2.csv")
# Drop the first column and normalize the remaining data
labels = data.iloc[:,0]
data_values = data.drop(columns=['Elements'])
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_values)

# Perform PCA with 2 components
pca = FastICA(n_components=2)
pca_data = pca.fit_transform(scaled_data)

# Perform Kernel PCA with 2 components
kpca = TSNE(n_components=2 )
kpca_data = kpca.fit_transform(scaled_data)

# Convert PCA and KPCA results to dataframes for easy handling
pca_df = pd.DataFrame(pca_data, columns=['PC1', 'PC2'])
kpca_df = pd.DataFrame(kpca_data, columns=['PC1', 'PC2'])

clustering = KMeans(n_clusters=5).fit(pca_df)
print()
final = pd.concat([labels, pd.DataFrame(pca_df),pd.DataFrame(clustering.labels_) ], axis=1)
print(final)
# Plot the clustered KPCA data
plt.figure(figsize=(12, 6), dpi=500)
plt.scatter(pca_df['PC1'], pca_df['PC2'], c=clustering.labels_, cmap='viridis', marker='o')
for i, txt in enumerate(labels):
    plt.annotate(txt, (pca_df['PC1'][i]+(random.random()*.12)-0.05, pca_df['PC2'][i]+(random.random()*.12)-0.05), fontsize=8, alpha=0.7)
plt.xlabel('Win Rate')
plt.ylabel('Pick Rate')
plt.suptitle('Unite Meta Analysis')
plt.title('Win Rate, Pick Rate Visualized')
plt.grid(True)
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'raw2.csv'